In [1]:
# your other imports here ...
import sys, os
import pandas as pd
import matplotlib.pyplot as plt

# TODO: replace with your path/to/ninarow
ninarowdir = os.path.dirname(os.getcwd())
modelfitdir = ninarowdir + "/model_fitting/"
# os.listdir(modelfitdir)

# sets the import path to the model-fitting directory
sys.path.insert(0, modelfitdir)
from parsers import *
# from new_fit import *
from utils import *
from utils import show
from tqdm import tqdm
import random

# WARNING: %load_ext autoreload and %autoreload 2 may interfere with 
# the Multi-threading processes!
%load_ext autoreload
%autoreload 2

In [4]:
df = pd.read_csv("../data/data.csv")
df["response_time"] = np.arange(len(df))
df = df[:50]
df.head()
make_splits(df, 5, "../data");

Saving split1 to ../data/1.csv
Saving split2 to ../data/2.csv
Saving split3 to ../data/3.csv
Saving split4 to ../data/4.csv
Saving split5 to ../data/5.csv


In [5]:
data_path = "../data"
output_path = "../data/out"
n_splits = 5
fold_number = 1
threads = 1
random_sample = False
verbose = True

print(f"Building output directory at {output_path}")
os.makedirs(output_path, exist_ok = True)

# first, we have to check to see if all the splits are there ...
assert np.all([f"{i + 1}.csv" in os.listdir(data_path) for i in range(n_splits)])
print("Detected splits in this directory. Loading splits ...")

# then we read them in
splits = [pd.read_csv(f"{data_path}/{i + 1}.csv") for i in range(n_splits)]

# we convert every row of our CSV to a "CSVMove object" using df_to_CSVMove - we do so for all the splits
# CSVMove is a class that is defined in the parsers.py file 
fold_data = [[csvmove for csvmove in df_to_CSVMove(split, warn = True)] for split in splits]

Building output directory at ../data/out
Detected splits in this directory. Loading splits ...


In [6]:
fold_data[0]

[545259552	6291456	White	1048576	45.0	1	1,
 8396800	4194304	White	2097152	33.0	1	1,
 2105344	4194304	White	16	4.0	1	1,
 8921600	7340032	White	4096	35.0	1	1,
 536870944	4194304	White	2097152	44.0	1	1,
 2684354560	4194304	White	2097152	7.0	1	1,
 8388608	0	White	2097152	40.0	1	1,
 8396808	3145728	White	524288	42.0	1	1,
 17197699080	14688256	White	16	23.0	1	1,
 1073741824	0	White	2097152	9.0	1	1]

In [7]:
import model
from time import time
# from utils import *

random.seed(10)
model.initialize_thread_pool(1, manual_seed = 10)
model_fitter =  model.Fitter(model.DefaultModel(), threads = 1, verbose = True)

tick = time()
# q = model.cross_validate(model.DefaultModel(), splits, leave_out_idx = 4, threads = 1)
model_fitter.fit(pd.concat([splits[0], splits[2], splits[3], splits[4]]))
print(time() - tick)

Setting manual seed 10 for single-thread
Thread 0: Base Seed 10, Seed: 10, Random Number: 601088376405717203

[Preprocessing] Initial log-likelihood estimation


100%|██████████| 10/10 [00:38<00:00,  3.86s/it]


	[0] NLL: 78.2633 Params: [2.001, 0.02, 0.2, 0.05, 1.2, 0.801, 1.001, 0.4, 3.501, 5.0]
Beginning optimization of a STOCHASTIC objective function

 Iteration    f-count      E[f(x)]        SD[f(x)]           MeshScale          Method              Actions
     0           1         78.2633             nan               1                                  
	[1] NLL: 138.1089 Params: [1.053, 0.68, 0.147, 0.292, 0.539, 0.952, -1.899, -1.372, -4.072, -3.799]


/Users/jordanlei/Github/ninarow/.venv/lib/python3.13/site-packages/pybads/bads/bads.py:34: SyntaxWarning: invalid escape sequence '\m'
  """


KeyboardInterrupt: 

In [8]:
import model_fit
from time import time

class DotDict(dict):
    """A dictionary that supports dot notation access."""
    def __getattr__(self, name):
        if name in self:
            return self[name]
        raise AttributeError(f"'{self.__class__.__name__}' object has no attribute '{name}'")
    
    def __setattr__(self, name, value):
        self[name] = value


args = DotDict({"random_sample": random_sample, 
    "verbose": verbose, 
    "threads": threads})

random.seed(10)
model_fit.initialize_thread_pool(1, manual_seed = 10)
model_fitter =  model_fit.ModelFitter(model_fit.DefaultModel(), args)

tick = time()
model_fitter.fit_model(fold_data[0] + fold_data[2] + fold_data[3] + fold_data[4])
print(time() - tick)

Setting manual seed 10 for single-thread
Thread 0: Base Seed 10, Seed: 10, Random Number: 601088376405717203

Beginning model fit pre-processing: log-likelihood estimation


100%|██████████| 40/40 [00:00<00:00, 2833.70it/s]


	[0] NLL: 78.2633 Params: [2.001, 0.02, 0.2, 0.05, 1.2, 0.801, 1.001, 0.4, 3.501, 5.0]
Beginning optimization of a STOCHASTIC objective function

 Iteration    f-count      E[f(x)]        SD[f(x)]           MeshScale          Method              Actions
     0           1         78.2633             nan               1                                  
	[1] NLL: 138.1089 Params: [1.053, 0.68, 0.147, 0.292, 0.539, 0.952, -1.899, -1.372, -4.072, -3.799]


KeyboardInterrupt: 

In [ ]:
fold_data[0] + fold_data[2] + fold_data[3] + fold_data[4]

In [ ]:
import model_new
from time import time

random.seed(10)
model_new.initialize_thread_pool(1, manual_seed = 10)
model_fitter =  model_new.ModelFitter(model_new.DefaultModel(), random_sample = random_sample, verbose = verbose, threads = threads)

tick = time()
model_fitter.fit_model(fold_data[0] + fold_data[2] + fold_data[3] + fold_data[4])
print(time() - tick)

In [ ]:
pd.concat([splits[0], splits[2], splits[3], splits[4]])

In [ ]:
fold_data[0] + fold_data[2] + fold_data[3] + fold_data[4]

In [42]:
import copy
import random

cutoff = 3.5
trials = fold_data[fold_number - 1]
expt_factor = 1.0

success_threshold = 1 # when "success_count" reaches this value, the tracker is considered done


n_trials = len(trials)
cumulative_cutoff = n_trials * cutoff

# trackers keeps track of the attempts and successes for IBS
data = dict(zip(trials, [{"attempt_count": 0, "success_count": 0, "log_likelihood": 0}] * n_trials))
dataloader = UltraDict(data, full_dump_size=n_trials*1024*1024, shared_lock = True) # buffer size is set to n_trials megabytes

global Lexpt
Lexpt.value =  n_trials * expt_factor


In [ ]:
scaling_factor = expt_factor / success_threshold

for _ in range(1):

    # incomplete_trials keeps track of all the trials that haven't yet reached the threshold to be considered "done"
    incomplete_trials = list(filter(lambda x: x[1]["success_count"] < success_threshold, trackers.items()))

    # if there are no more incomplete trials or if the Lexpt value is greater than the cutoff, we're done
    if not incomplete_trials or Lexpt.value > cutoff: 
        break

    # choose a trial at random, and get its tracker
    data, tracker = copy.deepcopy(random.choice(incomplete_trials))

    # accumulates all the changes we need to make to the expected log likelihood
    batch_delta_expected_log_likelihood = 0

    while tracker["success_count"] < success_threshold: 
        predicted_move = model.predict(data.board)
        actual_move = data.move.board_position

        # increment the attempt count
        tracker["attempt_count"] += 1

        # calculate the change in expected log likelihood
        # according to IBS, L = sum_k 1/k
        # where k here is the attempt count
        delta_expected_log_likelihood = -scaling_factor * 1/tracker["attempt_count"]

        # accumulate this change to the batch
        batch_delta_expected_log_likelihood += delta_expected_log_likelihood

        # update the log likelihood for this trial
        tracker["log_likelihood"] += delta_expected_log_likelihood

        # register a hit! and we're done with this trial
        if predicted_move == actual_move:
            # increment the success count and reset the attempts to 0
            tracker["success_count"] += 1
            tracker["attempt_count"] = 0
            with trackers.lock: 
                if tracker["success_count"] == trackers[data]["success_count"]:
                    trackers[data] = tracker
                    Lexpt.value += batch_delta_expected_log_likelihood
            break
        
        # if the expected log likelihood is greater than the cutoff, we're done
        if Lexpt.value + batch_delta_expected_log_likelihood > cutoff: 
            with trackers.lock: 
                Lexpt.value += batch_delta_expected_log_likelihood
            break

        




        
        delta_expected_log_likelihood = -scaling_factor * 1/tracker["attempt_count"]
        tracker["log_likelihood"] += delta_expected_log_likelihood
    
        


        



        # delta_expected_log_likelihood



